In [1]:
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go 

In [2]:
def create_scaled_spline_with_date(control_points, date_range=('2022-01-01', '2022-12-31'), y_range=(0, 1)):
    # Convert date range to pandas datetime for calculations
    start_date, end_date = pd.to_datetime(date_range[0]), pd.to_datetime(date_range[1])

    # Calculate total days for normalization
    total_days = (end_date - start_date).days

    # Sort control points by x value
    control_points.sort(key=lambda point: point[0])
    xs, ys = zip(*control_points)

    # Create spline
    spline = interp1d(xs, ys, kind='quadratic')

    # Generate x values for each day in the date range
    x = np.linspace(0, 1, num=total_days)

    # Generate y values (spline)
    y = spline(x)

    # Apply Min-Max scaling to y values
    y_min, y_max = np.min(y), np.max(y)
    y = (y - y_min) / (y_max - y_min)  # Min-Max scaling

    # Scale y values to provided y range
    y = y * (y_range[1] - y_range[0]) + y_range[0]

    # Convert normalized x values back to dates
    x_dates = [start_date + pd.Timedelta(days=val*total_days) for val in x]

    x_dates = [date.date() for date in x_dates]

    # Create a pandas Series with dates as the index
    spline_series = pd.Series(y, index=x_dates)

    return spline_series

In [3]:
date_range = ('2022-01-01', '2022-12-31')

y_range_1 = (0, 5)

control_points_1 = [(0, 0), (0.5, 0.9), (1, 1)]

series_1 = create_scaled_spline_with_date(control_points_1, date_range, y_range_1)

In [4]:
fig = px.line(series_1)
fig.show()

In [8]:
y_range_2 = (0, 10)

control_points_2 = [(0, 0), (0.3, 0.9), (0.4,0.9),(1, 1)]

series_2 = create_scaled_spline_with_date(control_points_1, date_range, y_range_1)

fig = px.line(series_2)
fig.show()

In [9]:
df = pd.concat({
    "s_1": series_1,
    "s_2": series_2
}, axis=1)

df

,s_1,s_2
2022-01-01,0.000000,0.000000
2022-01-02,0.033848,0.148450
2022-01-03,0.067581,0.295681
2022-01-04,0.101199,0.441692
2022-01-05,0.134702,0.586483
...,...,...
2022-12-26,4.764106,9.946292
2022-12-27,4.756684,9.959595
2022-12-28,4.749147,9.972981
2022-12-29,4.741495,9.986449


In [11]:
fig = px.line(df)
fig.show()

In [13]:
y_range_3 = (0, 10)

control_points_3 = [(0, 0.9), (0.3, 0.3), (0.4,0.2), (1, 1)]

series_3 = create_scaled_spline_with_date(control_points_3, date_range, y_range_3)

fig = px.line(series_3)
fig.show()

In [16]:
df = pd.concat([df, pd.DataFrame(series_3)], axis=1)
fig = px.line(df)
fig.show()